이 장에서는 함수 객체를 이용해 전략(strategy) 패턴을 리팩토링하고, 명령(command) 패턴을 단순화하는 방법을 설명한다.

# 6.1 사례: 전략 패턴의 리팩토링

* 전략 패턴은 파이썬에서 함수를 일급 객체로 사용하면 더욱 간단해질 수 있는 디자인 패턴

## 6.1.1 고전적인 전략

전략 패턴이란 :

> 일련의 알고리즘을 정의하고 각각을 하나의 클래스 안에 넣어서 교체하기 쉽게 만드는 패턴

전략 패턴의 구성 요소 :

* 콘텍스트(context) :  일부 계산을 서로 다른 알고리즘을 구현하는 교환 가능한 컴포넌트에 위임함으로써 서비스를 제공
* 전략(strategy) : 여러 알고리즘을 구현하는 컴포넌트의 공통된 인터페이스
* 구체적인 전략(concrete strategy) : 전략의 구상 서브클래스 중 하나

<img src='https://img1.daumcdn.net/thumb/R720x0.q80/?scode=mtistory2&fname=http%3A%2F%2Fcfile8.uf.tistory.com%2Fimage%2F2458DE3C52DFCAF121A3FD'>

### 전략 패턴 예제

온라인 상점이 다음과 같은 할인 규칙을 갖고 있다고 가정(하나의 주문에는 하나만 적용)

* 충성도 포인트가 1,000점 이상인 고객은 전체 주문에 대해 5% 할인 적용
* 하나의 주문에서 20개 이상의 동일 상품을 구입하면 해당 상품에 대해 10% 할인 적용
* 서로 다른 상품을 10종류 이상 주문하면 전체 주문에 대해 7% 할인 적용

이를 구현한 코드는 다음과 같다. 이 예제에서는 주문 객체를 생성하기 전에 시스템이 할인 전략을 선택해서 `Order` 생성자에 전달

In [90]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [91]:
jack = Customer('Jack',100)
jack

Customer(name='Jack', fidelity=100)

In [92]:
class LineItem:
    """구매할 물품/갯수 생성해서 총 가격을 반환"""
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

In [93]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 할인 객체
        
    def total(self):
        """__total 속성이 없으면 전체 계산할 값을 계산"""
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        """할인금액 차감"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self) # self = Order 객체
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    

이제 전략 구조를 생성하자.

In [94]:
class Promotion(ABC): # strategy : abstract base class 
    """할인 혜택 클래스들의 형태를 선언"""
    
    @abstractmethod # 이 클래스를 상속하는 클래스는 반드시 이 메서드를 선언해야함
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환"""
        pass

이렇게 정의한 전략의 구체적인 전략을 생성하자.

In [95]:
class FidelityPromo(Promotion):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion): 
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount
    
class LargeOrderPromo(Promotion):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

실제 연산의 예를 들어보자.

In [96]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]

In [97]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

Joe는 충성도가 0이므로 충성도 할인은 없다.

In [98]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

Ann은 충성도가 1000점 이상이므로 5% 할인(2.1달러)

In [99]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [100]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

바나나가 30개 이상이므로 10% 할인(15달러의 10%인 1.5달러)

In [101]:
long_order = [LineItem(str(item_code), 1, 1.0)
             for item_code in range(10)] # 물품 종류가 10개

In [102]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

10개 이상을 구입했으므로 10% 할인 적용

In [103]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

물품의 갯수가 3개이므로 할인 없음

위의 예제는 제대로 작동하지만, 파이썬에서 함수를 객체로 사용하면 더 적은 코드로 동일한 기능을 구현할 수 있다.

## 6.1.2 함수지향 전략

위 예제에서 구체적인 전략들은 속성은 없고 `discount()`라는 메서드 하나만을 가진 클래스이므로, 이를 함수로 대신할 수 있다.

In [104]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 할인 객체
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self) # promotion은 함수 객체
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    

In [105]:
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order): 
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount
    
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

코드가 짧아졌으며 `Order` 객체를 사용하는 방법도 더 간단해졌다.

In [106]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [107]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [108]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [109]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

플라이웨이트란 :
> 여러 콘텍스트에서 동시에 사용할 수 있는 공유 객체

* 새로운 콘텍스트에서 동일 전략 객체를 반복해서 적용할 때는 새로 생성하는 비용을 줄이기 위해 플라이웨이트를 공유하는 것이 좋다. 
* 전략 객체가 내부 상태를 가지고 있지 않은 경우는 메서드를 하나만 구현하는 클래스를 만들기 보다는 일반 함수를 만드는 것이 좋다.

## 6.1.3 최선의 전략 선택하기: 단순한 접근법

주어진 `Order` 객체에 대해 적용할 수 있는 가장 좋은 할인 전략을 선택하는 '메타 전략'을 만든다고 가정해보자.

In [110]:
# Order(joe, long_order, best_promo)

In [111]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
# 함수들로 구현된 전략들의 리스트

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [112]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [113]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [114]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

위 코드는 제대로 작동하지만, 일부 코드가 중복되어 있어 다음과 같은 버그가 생길 여지가 있다. 

* 새로운 할인 전략을 추가하려면 함수를 코딩하고 이 함수를 `promos` 리스트에 추가해야 한다.
* 아니면 새로운 할인 함수를 `Order` 객체에 인수로 전달해서 작동시킬 수 있지만 이때 `best_promo()`는 새로운 할인 함수를 고려하지 않는다.

## 6.1.4 모듈에서 전략 찾기

파이썬은 모듈을 다루는 여러 함수를 표준 라이브러리에서 제공하며, `globals()` 함수도 그 중 하나이다.

> __globals()__ <br>
> 현재 전역 심벌 테이블을 나타내는 딕셔너리 객체를 반환. 이 딕셔너리는 언제나 현재 모듈에 대한 내용을 담고 있다.

In [115]:
globals?

Signature: globals()
Docstring:
Return the dictionary containing the current scope's global variables.

NOTE: Updates to this dictionary *will* affect name lookups in the current
global scope and vice-versa.
Type:      builtin_function_or_method


In [116]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractclassmethod', 'namedtuple', 'Customer', '_i2', 'jac', '_i3', '_3', '_i4', 'jack', '_4', '_i5', 'LineItem', '_i6', 'Order', '_i7', '_i8', 'abstractmethod', '_i9', 'Promotion', '_i10', '_i11', '_i12', 'FidelityPromo', '_i13', '_i14', 'BulkItemPromo', 'LargeOrderPromo', '_i15', 'joe', '_i16', 'ann', 'cart', '_i17', '_17', '_i18', '_18', '_i19', '_19', '_i20', '_20', '_i21', '_21', '_i22', 'banana_cart', '_i23', '_23', '_i24', '_24', '_i25', '_25', '_i26', 'long_order', '_i27', '_27', '_i28', '_28', '_i29', '_29', '_i30', '_30', '_i31', '_i32', '_i33', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i34', '_34', '_i35', '_35', '_i36', '_36', '_i37', '_37', '_i38', 'Heater', '_i39', 'heat', '_i40', '_i41', 'OKGoogle', '_i42', '_i43', 'Client', '_i44'

이 함수를 이용하면 `best_promo()`가 자동으로 다른 `*_promo()` 함수들을 찾아내게 할 수 있다.

In [145]:
globals()['_i100']

'Order(joe, banana_cart, BulkItemPromo())'

In [117]:
promos = [globals()[name] for name in globals()
         if name.endswith('_promo')
         and name != 'best_promo'] # 재귀호출을 피하기 위함
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

별도의 모듈을 만들고 `best_promo()`를 제외한 모든 프로모션 할인 함수를 그 모듈에 넣어서 적용할 수 있는 모든 할인 함수를 모으는 방법도 있다.

In [118]:
!cat promotions.py

def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0


In [119]:
import promotions, inspect

In [120]:
promos = [func for name, func in
         inspect.getmembers(promotions, inspect.isfunction)]

In [121]:
promos

[<function promotions.bulk_item_promo(order)>,
 <function promotions.fidelity_promo(order)>,
 <function promotions.large_order_promo(order)>]

`inspect.getmembers()` 함수는 조건식으로 걸러낸 객체의 속성들을 반환한다.

# 6.2 명령

함수를 인수로 전달하는 기법을 사용하면 명령(command) 디자인 패턴도 구현을 단순하게 만들 수 있다. 

* 명령 패턴 : 연산을 실행하는 객체(호출자, invoker)와 연산을 수행하는 객체(수신자, receiver)를 분리하는 패턴

### 명령 패턴 예제 

구글홈을 사용하는 사용자를 Client 클래스, 구글홈을 OKGoogle 클래스, 히터를 Heater 클래스로 정의

In [122]:
class Heater:
    def __init__(self):
        pass
    
    def powerOn(self):
        print("Heater on")

In [123]:
class OKGoogle:
    def __init__(self, heater):
        self.heater = heater
        
    def talk(self):
        self.heater.powerOn()

In [124]:
class Client:
    def __init__(self):
        pass
    
    def main(self):
        heater = Heater()
        okGoogle = OKGoogle(heater)
        okGoogle.talk()

In [125]:
client = Client()
client.main()

Heater on


램프를 켜는 기능을 추가하려면?

`Lamp` 클래스를 정의하고 `OKGoogle` 클래스에서도 `Lamp` 객체를 참조하도록 해야한다.

In [126]:
class Lamp:
    def __init__(self):
        pass
    
    def turnOn(self):
        print("Lamp on")

In [127]:
modes = ['heater', 'lamp']

class OKGoogle:
    def __init__(self, heater, lamp):
        self.heater = heater
        self.lamp = lamp
        
    def setMode(self, idx):
        self.mode = modes[idx]
        
    def talk(self):
        if self.mode == 'heater':
            self.heater.powerOn()
        elif self.mode == 'lamp':
            self.lamp.turnOn()

In [128]:
class Client:
    def __init__(self):
        pass
    
    def main(self):
        heater = Heater()
        lamp = Lamp()
        okGoogle = OKGoogle(heater, lamp)
        
        # 히터 켜짐
        okGoogle.setMode(0)
        okGoogle.talk()
        
        # 램프 켜짐
        okGoogle.setMode(1)
        okGoogle.talk()

In [129]:
client = Client()
client.main()

Heater on
Lamp on


`OKGoogle`에게 `mode` 설정을 통해 `mode='heater'`이면 히터, `mode='lamp'`이면 램프를 켜도록 설정함

그러나 기능이 많아질 수록 객체의 프로퍼티는 더욱 늘어나고 `talk()` 메서드의 분기가 늘어나게 된다. 또한 OCP("클래스나 모듈은 확장에는 열려 있어야 하고 변경에는 닫혀 있어야 한다") 원칙에도 위배된다.

이를 해결하기 위해, `OKGoogle`이 할 수 있는 기능들을 클래스로 만들어서(`HeaterOnCommand`, `LampOnCommand`) 각 기능들을 캡슐화하고, `OKGoogle` 클래스의 `talk()` 메서드에서 캡슐화한 `Command` 인터페이스의 메서드를 호출하도록 한다.

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile10.uf.tistory.com%2Fimage%2F997EC63E5C16561421D06C'>

먼저 인터페이스를 정의한다.

In [130]:
class Command(ABC):

    @abstractmethod
    def run(self):
        pass

`Heater`와 `Lamp`를 켜는 명령을 클래스화하여 `HeaterOnCommand`와 `LampOnCommand`를 정의한다.

In [131]:
class HeaterOnCommand(Command):
    def __init__(self, heater):
        self.heater = heater
        
    def run(self):
        self.heater.powerOn()

In [132]:
class Heater:
    
    def powerOn(self):
        print("Heater on")

In [133]:
class LampOnCommand(Command):
    def __init__(self, lamp):
        self.lamp = lamp
        
    def run(self):
        self.lamp.turnOn()

In [134]:
class Lamp:
    
    def turnOn(self):
        print("Lamp on")

`OKGoogle` 클래스의 `talk()` 메서드에서는 `Command` 인터페이스의 `run()` 메서드를 통해 명령을 실행한다.

In [135]:
class OKGoogle:
    
    def setCommand(self, command):
        self.command = command
        
    def talk(self):
        self.command.run()

마지막으로 `OKGoogle`을 사용하는 `Client` 클래스를 정의한다.

In [136]:
class Client:
    def __init__(self):
        pass
    
    def main(self):
        heater = Heater()
        lamp = Lamp()
        
        heaterOnCommand = HeaterOnCommand(heater)
        lampOnCommand = LampOnCommand(lamp)
        okGoogle = OKGoogle()
        
        # 히터를 켠다
        okGoogle.setCommand(heaterOnCommand)
        okGoogle.talk()
        
        # 램프를 켠다
        okGoogle.setCommand(lampOnCommand)
        okGoogle.talk()

In [137]:
client = Client()
client.main()

Heater on
Lamp on


그런데 이 구조에서 `HeaterOnCommand` 클래스의 `run()`을 콜러블로 변경하여 다음과 같이 할 수 있다.

In [138]:
class Command(ABC):

    @abstractmethod
    def __call__(self):
        pass

In [139]:
class HeaterOnCommand(Command):
    def __init__(self, heater):
        self.heater = heater
        
    def __call__(self):
        self.heater.powerOn()

In [140]:
class LampOnCommand(Command):
    def __init__(self, lamp):
        self.lamp = lamp
        
    def __call__(self):
        self.lamp.turnOn()

In [141]:
class OKGoogle:
    def __init__(self):
        pass
    
    def setCommand(self, command):
        self.command = command
        
    def talk(self):
        self.command()

In [142]:
class Client:
    def __init__(self):
        pass
    
    def main(self):
        heater = Heater()
        lamp = Lamp()
        
        heaterOnCommand = HeaterOnCommand(heater)
        lampOnCommand = LampOnCommand(lamp)
        okGoogle = OKGoogle()
        
        # 히터를 켠다
        okGoogle.setCommand(heaterOnCommand)
        okGoogle.talk()
        
        # 램프를 켠다
        okGoogle.setCommand(lampOnCommand)
        okGoogle.talk()

In [143]:
client = Client()
client.main()

Heater on
Lamp on


## 끗!